# New Section

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
%matplotlib inline
import matplotlib.pyplot as plt  # Matlab-style plotting
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')
import warnings
def ignore_warn(*args, **kwargs):
    pass
warnings.warn = ignore_warn #ignore annoying warning (from sklearn and seaborn)

from statistics import stdev
import math
from scipy import stats
from operator import add
from scipy.stats import norm, skew #for some statistics
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
#from fancyimpute import KNN
from sklearn import feature_selection
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, make_scorer
import os

from sklearn.model_selection import train_test_split  
import pickle


pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points

In [ ]:
from sklearn.linear_model import ElasticNet, Lasso, Ridge, LinearRegression
import sklearn.ensemble 
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_curve, auc

import tensorflow as tf

from keras.regularizers import l2

from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization

from keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import Callback
from keras import backend as K

import matplotlib.pyplot as plt
%matplotlib inline  

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

# Loading the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/My Drive/

/content/drive/My Drive


In [ ]:
data = pd.read_csv("Train.csv")
test = pd.read_csv("Test.csv")

In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/

/content/drive/My Drive/Colab Notebooks


In [ ]:
data_copy = data.copy()
test_copy= test.copy()
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.750,14056.000,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.950,13098.000,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.950,15044.000,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.830,15525.000,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.650,13952.000,35


In [ ]:
data.shape

(284780, 8)

# Null counts

In [ ]:
# Function to count the null values and their percentages in each column.
def fn_nulls_count(df):
    # Nulls counts in each column.
    nulls_counts = df.isnull().sum()
    # Convert above series into dataframe.
    nulls_counts_df = nulls_counts.to_frame(name = 'nulls_count')
    # Add a new column - to get the null counts percentage to the total rows.
    nulls_counts_df['nulls_percentage'] = round(100 * nulls_counts_df['nulls_count']/df.shape[0], 2)
    # Return the null counts dataframe.
    return nulls_counts_df

In [ ]:
fn_nulls_count(data)

,nulls_count,nulls_percentage
InvoiceNo,0,0.000
StockCode,0,0.000
Description,0,0.000
Quantity,0,0.000
InvoiceDate,0,0.000
UnitPrice,0,0.000
CustomerID,0,0.000
Country,0,0.000


In [ ]:
# function to check for Outliers and replacing with respective whiskers
def fn_outliers_replace(df):
    num_cols = df.select_dtypes(exclude =['object','category']).columns.tolist()
    col = df[num_cols].columns
    for i in range(0,df[col].shape[1]):
        Q0 = df[col[i]].quantile(0.01)
        Q1 = df[col[i]].quantile(0.25)
        Q3 = df[col[i]].quantile(0.75)
        Q4 = df[col[i]].quantile(1)
        IQR = Q3 - Q1
        Lower_whisker = Q1 - 1.5 * IQR
        Upper_Whisker = Q3 + 1.5 * IQR
         
    
        Outliers_counts = len((df[col[i]][df[col[i]] < Lower_whisker ]) + (df[col[i]][train[col[i]] > Upper_Whisker]))
        print("Outliers counts in", col[i], Outliers_counts)
    
        df.loc[(df[col[i]] < Lower_whisker),col[i]] = Lower_whisker
        df.loc[(df[col[i]] > Upper_Whisker),col[i]] = Upper_Whisker
    
    return ''

# Feature extraction

In [ ]:
#train feature extraction
import datetime as dt
frame = pd.to_datetime(data['InvoiceDate'], dayfirst = True) 
frame = pd.DataFrame([frame]).transpose() 
data['date']= frame 
data['month']= data['date'].dt.month
data['day']= data['date'].dt.day
data['hour']= data['date'].dt.hour
data['week']= data['date'].dt.week
data['year']= data['date'].dt.year

#Test feature extraction
frame1 = pd.to_datetime(test['InvoiceDate'], dayfirst = True) 
frame1 = pd.DataFrame([frame1]).transpose() 
test['date']= frame1
test['month']= test['date'].dt.month
test['day']= test['date'].dt.day
test['hour']= test['date'].dt.hour
test['week']= test['date'].dt.week
test['year']= test['date'].dt.year

# Binning

In [ ]:
#datatype conversion

cat_labels = ['InvoiceNo','StockCode','Description','Country','CustomerID', 'month', 'day', 'week','hour', 'year']
for label in cat_labels:
    data[label] = data[label].astype('category')

for label in cat_labels:
    test[label] = test[label].astype('category')

num_labels = ['Quantity']    

In [ ]:
#Dropping irrelevant columns
data.drop(["InvoiceNo","CustomerID","InvoiceDate",'StockCode',"date"],inplace=True,axis=1)
test.drop(["InvoiceNo","CustomerID","InvoiceDate","date",'StockCode'],inplace=True,axis=1)

In [ ]:
test['prediction']=0
#data.drop('prediction',axis=1,inplace=True)
#test.drop('predictions',axis=1,inplace=True)


In [ ]:
#  Q=[]
#  for i in range(1,21,):
#    Q.append(data['StockCode'].quantile(0.05*i))
#  Q


[262.0,
 418.0,
 627.0,
 829.0,
 939.0,
 1066.0,
 1228.0,
 1325.0,
 1432.0,
 1521.0,
 1613.0,
 1756.0,
 1863.0,
 2026.0,
 2106.0,
 2196.0,
 2594.0,
 2802.0,
 3146.0,
 3683.0]

In [ ]:
len(labels)

18

In [ ]:
# # bins=[-1, 418.0, 1066.0, 1521.0, 2026.0, 3684.0]
# # labels=['0-418', '419-1066', '1067-1521', '1522-2026', '2027-3690']

# bins=[-1, 262.0,       418.0,       627.0,     829.0,       939.0,      1066.0,    1228.0,     1325.0,    1432.0,   1521.0,   1613.0,       1756.0,      1863.0,     2026.0,     2106.0,     2196.0,      2594.0,     2802.0,     3146.0, 3683.0]
# labels=['0-262','262-418', '419-627', '627-829', '829-939', '939-1066' ,'1066-1228', '1228-1325', '1325-1432','1432-1521' ,'1521-1613','1613-1756', '1756-1863','1863-2026','2026-2106','2106-2196', '2196-2594', '2594-2802','2802-3146','3146-3690']
# data['stock_bin']=pd.cut(data['StockCode'],bins=bins,labels=labels)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
num_attr = ['Quantity']
cat_attr = ['Description','month','day','hour','week','Country','year']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

In [ ]:
test_pred = base_mod.predict(q.drop('prediction',axis=1))
  test_pred=pd.DataFrame(list(test_pred),columns=['UnitPrice'],index=index2)
  temp=pd.concat([temp,test_pred])
  #test['prediction']=test_pred

(3819, 3563)

In [ ]:
#Splitting data into 10 dataframes based on Stock_bin
from math import sqrt
data_dict={}
temp=pd.DataFrame()
for (name,df),(test_name,test_df) in zip(data.groupby('Description',as_index=False),test.groupby('Description',as_index=False)):
   
  base_model = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', DecisionTreeRegressor())])
  df_X=df.drop('UnitPrice',axis=1)
  df_y=df.UnitPrice
  base_model.fit(df_X, df_y)
  train_pred = base_model.predict(df_X)
  data_dict[name]=base_model
  rmse_error=sqrt(mean_squared_error(df_y,train_pred))
  print(rmse_error)
  test_pred = data_dict[name].predict(test_df.drop('prediction',axis=1))
  #y['prediction']=test_pred
  index2=test_df.index.values
  test_pred=pd.DataFrame(list(test_pred),columns=['UnitPrice'],index=index2)
    temp=pd.concat([temp,test_pred])
  print("Rsquared for the base model is : ({:.4f})".format(base_model.score(df_X,df_y)))


8.22293433142597e-16
Rsquared for the base model is : (1.0000)


In [ ]:
temp.sort_index(axis=0)

,UnitPrice
51,0.650
197,1.250
285,1.650
416,0.650
455,0.650
...,...
121367,12.750
121518,7.950
121888,0.650
121960,0.650


In [ ]:
temp=temp.sort_index(axis=0)
temp.drop_duplicates(keep='first', inplace=True)
temp

,UnitPrice
0,12.500
1,1.650
2,0.360
4,3.950
5,2.550
...,...
110079,635.010
112473,1.080
114988,2.890
116749,4.980


In [ ]:
temp

,UnitPrice
0,12.500
1,1.650
2,0.360
4,3.950
5,2.550
...,...
110079,490.250
112473,1.080
114988,2.890
116749,4.980


In [ ]:
# bins=[-1, 418.0, 1066.0, 1521.0, 2026.0, 3684.0]
# labels=['0-418', '419-1066', '1067-1521', '1522-2026', '2027-3690']
#test['stock_bin']=pd.cut(test['StockCode'],bins=bins,labels=labels)

In [ ]:
#Splitting data into 10 dataframes based on Stock_bin
test_list = [y for x, y in test.groupby('Description', as_index=False)]
test_list

[      StockCode Description  Quantity Country  ... hour week  year prediction
 20695      2692           0         1      35  ...   12   41  2011          0
 26204      2692           0        12      35  ...    9   49  2011          0
 27211      2692           0         3      35  ...   12   14  2011          0
 48797      2692           0         4      35  ...   13   44  2011          0
 50821      2692           0         1      35  ...   13   19  2011          0
 64910      2692           0         1      35  ...   12   46  2011          0
 69068      2692           0         2      35  ...   16   24  2011          0
 74642      2692           0        12      35  ...   10   47  2011          0
 79779      2692           0         6      35  ...   16   47  2011          0
 96907      2692           0         1      35  ...   13   16  2011          0
 99928      2692           0         1      35  ...   13    4  2011          0
 
 [11 rows x 10 columns],
        StockCode Descrip

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

from sklearn.preprocessing import StandardScaler
num_attr = ['Quantity']
cat_attr = ['Description','month','day','hour','week','Country','year','StockCode']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attr),
        ('cat', categorical_transformer, cat_attr)])

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
import xgboost
#Training model on bins
from math import sqrt
from sklearn.model_selection import GridSearchCV
dict_models={}
rmse={}
temp=pd.DataFrame()
for i, j in zip(range(len(df_list)), range(len(test_list))):
  index2=index1[j]
  test_df=test_list[j]
  df=df_list[i]
  df_X= df.drop(['UnitPrice'],axis=1)
  df_y= df['UnitPrice']
  # base_model = Pipeline(steps=[('preprocessor', preprocessor),
  #                       ('classifier', GradientBoostingRegressor())])
  # param_grid = {
  #    'classifier__n_estimators':[100,500],
  #    'classifier__max_depth': [10,20],
  #   'classifier__learning_rate' : [0.001,0.015,0.01],
  #   'classifier__max_features' : ['sqrt']
  #   }
  # gbm_model = GridSearchCV(base_model, param_grid= param_grid, cv= 5,n_jobs=-1)
  # base_model = Pipeline(steps=[('preprocessor', preprocessor),
  #                      ('classifier', GradientBoostingRegressor())])
  # base_model = Pipeline(steps=[('preprocessor', preprocessor),
  #                       ('classifier', RandomForestRegressor())])
  # base_model = Pipeline(steps=[('preprocessor', preprocessor),
  #                     ('classifier', xgboost.XGBRegressor())])
  
  base_model = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', DecisionTreeRegressor())])
  #base_model = Pipeline(steps=[('preprocessor', preprocessor),
  #                   ('classifier', LinearRegression())]) #30
  base_mod = base_model.fit(df_X, df_y)
  train_pred = base_mod.predict(df_X)
  
  test_pred = base_mod.predict(test_df.drop('prediction',axis=1))
  test_pred=pd.DataFrame(list(test_pred),columns=['UnitPrice'],index=index2)
  temp=pd.concat([temp,test_pred])
  #test_df['prediction']=test_pred
  rmse_error=sqrt(mean_squared_error(df_y,train_pred))
  print(rmse_error)
  print("Rsquared for the base model is : ({:.4f})".format(base_model.score(df_X,df_y)))
  #key=df.stock_bin.unique()
  #dict_models[str(key[0])]=base_mod
  #rmse[str(key[0])]=rmse_error
  #rmse[str(df['StockCode'])]=rmse_error


Streaming output truncated to the last 5000 lines.
0.0
Rsquared for the base model is : (1.0000)
0.0
Rsquared for the base model is : (1.0000)
2.220446049250313e-16
Rsquared for the base model is : (0.0000)
0.0
Rsquared for the base model is : (nan)
2.220446049250313e-16
Rsquared for the base model is : (0.0000)
2.293266818639604e-16
Rsquared for the base model is : (1.0000)
0.0
Rsquared for the base model is : (nan)
0.0
Rsquared for the base model is : (nan)
0.0
Rsquared for the base model is : (1.0000)
0.0
Rsquared for the base model is : (1.0000)
0.0
Rsquared for the base model is : (1.0000)
0.0
Rsquared for the base model is : (1.0000)
1.3322676295501878e-15
Rsquared for the base model is : (-8.0000)
4.52772401965236e-16
Rsquared for the base model is : (1.0000)
4.031894017494064e-15
Rsquared for the base model is : (1.0000)
1.9984014443252818e-15
Rsquared for the base model is : (-19.2500)
2.8285716740879595e-15
Rsquared for the base model is : (1.0000)
2.4494461288629646e-16
Rsqu

In [ ]:
temp

,UnitPrice
0,12.500
1,1.650
2,0.360
4,3.950
5,2.550
...,...
110079,490.250
112473,1.080
114988,2.890
116749,4.980


In [ ]:
from google.colab import files

temp.to_csv('Desc1.csv')
files.download('Desc1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>